In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

In [30]:
roberta_path = '/root/projects/prompt_tuning_with_constraint_ICLR/results/FacebookAI'

bert_path = '/root/projects/prompt_tuning_with_constraint_ICLR/results/'

In [23]:
os.listdir(result_path)

['roberta-large', 'roberta-base']

In [51]:
df = pd.read_csv(f'/root/projects/prompt_tuning_with_constraint_ICLR/results/FacebookAI/roberta-base/newprompt/trec_2__trec-gamma-5e-07-lr-0.001-lr_LM-0.001-epoch-3-num_of_init_text-1-seed-50similarityNone-layer--3.csv')
df.head()

,epoch,eval_loss,eval_accuracy,layer,prompt
0,1.0,1.567212,0.444,-2,trec_2
1,2.0,1.206155,0.546,-2,trec_2
2,3.0,1.030909,0.674,-2,trec_2
3,1.0,1.539009,0.460,-1,trec_2
4,2.0,1.112942,0.654,-1,trec_2


In [56]:
df['layer'].unique()

array([-2, -1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [68]:
def read_csv(file_name):
    df = pd.read_csv(file_name)
    u = len(df['layer'].unique())
    return df, u
from collections import defaultdict

task_baselinse = {
    'SST-2': 0.878,
    'sst-5': 0.477,
    'subj': 0.932,
    'trec': 0.804,
}
def read_files(model_name, task_name, result_path=bert_path, multi=1):
    file_names = []
    all_files = os.listdir(f'{result_path}/{model_name}')
    layer_num = 0
    file_count = 0
    all_layer_dict = {}
    dict2 = defaultdict(list)

    for file in all_files:
        filename = f'{result_path}/{model_name}/{file}'
        if task_name not in file:
            continue
        df, layer_num = read_csv(filename)
        print(df.shape, layer_num)
        file_count += 1
        for i in range(0, layer_num):
            i2 = (i+1)*multi-1
            delta = df['eval_accuracy'][i2] - task_baselinse[task_name]#- df['eval_accuracy'][0]
            # print(i2, i , delta)
            dict2[df['layer'][i2]].append(delta - task_baselinse[task_name])
            if not all_layer_dict.get(df['layer'][i2]):
                all_layer_dict[df['layer'][i2]] = delta
            else:
                if delta > all_layer_dict[df['layer'][i2]]:
                    all_layer_dict[df['layer'][i2]] = delta
    
    for k, v in dict2.items():
        dict2[k] = np.std(v)

    return all_layer_dict, file_count, dict2


In [69]:
read_files('bert-base-uncased', 'SST-2')

(2, 5) 2
(14, 5) 14
(14, 5) 14
(14, 5) 14
(14, 5) 14
(14, 5) 14


({-2: 0.006174311926605447,
  -1: -0.0007064220183485848,
  0: -0.0007064220183485848,
  1: -0.006440366972477074,
  2: -0.0018532110091743492,
  3: 0.0015871559633027221,
  4: -0.0018532110091743492,
  5: -0.0007064220183485848,
  6: -0.004146788990825656,
  7: -0.0007064220183485848,
  8: -0.0075871559633027275,
  9: -0.006440366972477074,
  10: -0.0075871559633027275,
  11: -0.0075871559633027275},
 6,
 defaultdict(list,
             {-2: 0.024452883167766764,
              -1: 0.02296840823891848,
              0: 0.006380930157451167,
              1: 0.004789131426105762,
              2: 0.0069794147039921655,
              3: 0.007410777257524537,
              4: 0.0073751996288135375,
              5: 0.00704692270446662,
              6: 0.0037407124840137793,
              7: 0.00633957567022259,
              8: 0.004506165757887283,
              9: 0.003932667018229871,
              10: 0.002655008463942711,
              11: 0.0038652063179707968}))

In [33]:
read_files('bert-base-uncased', 'sst-5')

({-2: -0.044665758401453204,
  -1: -0.041032697547683905,
  0: -0.0482988192552225,
  1: -0.052840145322434195,
  2: -0.05828973660308806,
  3: -0.05465667574931876,
  4: -0.05102361489554946,
  5: -0.066464123524069,
  6: -0.05828973660308806,
  7: -0.047390554041780164,
  8: -0.0591980018165304,
  9: -0.05465667574931876,
  10: -0.06010626702997279,
  11: -0.05374841053587648},
 5,
 defaultdict(list,
             {-2: 0.057874670803748036,
              -1: 0.03998595695913344,
              0: 0.035337937117606806,
              1: 0.0363151647465612,
              2: 0.03428674113131578,
              3: 0.03466195197177382,
              4: 0.034716172987185705,
              5: 0.03464766918071135,
              6: 0.031413898862048824,
              7: 0.03489157295402222,
              8: 0.034010378707635326,
              9: 0.036699309861360635,
              10: 0.031675415635379095,
              11: 0.033913217102675176}))

In [34]:
read_files('bert-base-uncased', 'subj')

({-2: 0.013499999999999956,
  -1: 0.010499999999999954,
  0: 0.010999999999999899,
  1: 0.010999999999999899,
  2: 0.012499999999999956,
  3: 0.011499999999999955,
  4: 0.009499999999999953,
  5: 0.010499999999999954,
  6: 0.010499999999999954,
  7: 0.010499999999999954,
  8: 0.008499999999999952,
  9: 0.008999999999999897,
  10: 0.010999999999999899,
  11: 0.0119999999999999},
 5,
 defaultdict(list,
             {-2: 0.01315446692192428,
              -1: 0.0025377155080898994,
              0: 0.012095453691366826,
              1: 0.012019151384353203,
              2: 0.012452309022827837,
              3: 0.011565465835840758,
              4: 0.0119899958298575,
              5: 0.01155248890932166,
              6: 0.011659331027121564,
              7: 0.01148738438461948,
              8: 0.011289818421923338,
              9: 0.01144290172989349,
              10: 0.011842297074469944,
              11: 0.012255610959882797}))

In [24]:
read_files('bert-base-uncased', 'trec')

({-1: 0.20400000000000001,
  0: 0.04999999999999999,
  1: 0.04999999999999999,
  2: 0.04999999999999999,
  3: 0.04999999999999999,
  4: 0.04999999999999999,
  5: 0.04999999999999999,
  6: 0.04999999999999999,
  7: 0.04999999999999999,
  8: 0.04999999999999999,
  9: 0.04999999999999999,
  10: 0.04999999999999999,
  11: 0.04999999999999999},
 5)

In [26]:
read_files('roberta-base', 'SST-2')

({-2: 0.039431192660550396,
  -1: 0.03484403669724756,
  0: 0.035990825688073436,
  1: 0.03369724770642202,
  2: 0.0314036697247706,
  3: 0.03484403669724756,
  4: 0.039431192660550396,
  5: 0.035990825688073436,
  6: 0.03369724770642202,
  7: 0.035990825688073436,
  8: 0.03828440366972474,
  9: 0.035990825688073436,
  10: 0.035990825688073436,
  11: 0.0371376146788992},
 1,
 defaultdict(list,
             {-2: 0.0,
              -1: 0.0,
              0: 0.0,
              1: 0.0,
              2: 0.0,
              3: 0.0,
              4: 0.0,
              5: 0.0,
              6: 0.0,
              7: 0.0,
              8: 0.0,
              9: 0.0,
              10: 0.0,
              11: 0.0}))

In [27]:
read_files('roberta-base', 'sst-5')

({-2: -0.0737302452316076,
  -1: -0.026500454132606766,
  0: -0.0737302452316076,
  1: -0.01741780199818349,
  2: -0.0482988192552225,
  3: -0.040124432334241567,
  4: -0.037399636693914606,
  5: -0.031041780199818403,
  6: -0.028316984559491387,
  7: -0.07282198001816537,
  8: -0.0591980018165304,
  9: -0.09280381471389648,
  10: -0.023775658492279805,
  11: -0.06464759309718437},
 2,
 defaultdict(list,
             {-2: 0.06939764394148253,
              -1: 0.08054536789872241,
              0: 0.06980589213780018,
              1: 0.08941992880965925,
              2: 0.07245783424182924,
              3: 0.08272141744347372,
              4: 0.08608897810043806,
              5: 0.08507794587949648,
              6: 0.08196670717655058,
              7: 0.07044115780232979,
              8: 0.07307255655119342,
              9: 0.05870794772458606,
              10: 0.08605490032449603,
              11: 0.06510993995642315}))

In [28]:
read_files('roberta-base', 'subj')

({-2: 0.011499999999999955,
  -1: 0.0129999999999999,
  0: 0.013999999999999901,
  1: 0.015999999999999903,
  2: 0.009999999999999898,
  3: 0.012499999999999956,
  4: 0.0129999999999999,
  5: 0.0129999999999999,
  6: 0.009499999999999953,
  7: 0.013499999999999956,
  8: 0.010499999999999954,
  9: 0.013499999999999956,
  10: 0.011499999999999955,
  11: 0.015499999999999958},
 3,
 defaultdict(list,
             {-2: 0.007675719293112976,
              -1: 0.006357410374253518,
              0: 0.0127401923062409,
              1: 0.011855612829185807,
              2: 0.013795127964450194,
              3: 0.016781479540122664,
              4: 0.008757932911874145,
              5: 0.012591884511682727,
              6: 0.014174507634012084,
              7: 0.01217494010936672,
              8: 0.01474128744565939,
              9: 0.011440668201153659,
              10: 0.010678704145270728,
              11: 0.01233445940805227}))

In [77]:
read_files('roberta-base/newprompt', 'sst-5', roberta_path, multi=3)

(42, 5) 14
(81, 5) 14


({-2: -0.016509536784741097,
  -1: -0.0019772933696639017,
  0: -0.016509536784741097,
  1: -0.00016076294277928005,
  2: -0.03285831062670297,
  3: -0.031950045413260686,
  4: -0.02559218891916437,
  5: -0.021959128065395073,
  6: -0.027408719346049104,
  7: -0.043757493188010865,
  8: -0.0419409627611263,
  9: -0.031950045413260686,
  10: -0.03376657584014536,
  11: -0.03376657584014536},
 2,
 defaultdict(list,
             {-2: 0.002724795640326988,
              -1: 0.013272970158512285,
              0: 0.0021408016384697483,
              1: 0.01798273376314565,
              2: 0.006422404915409193,
              3: 0.005994244587715192,
              4: 0.0004281603276939497,
              5: 0.015894641235240675,
              6: 0.0,
              7: 0.0,
              8: 0.0,
              9: 0.0,
              10: 0.0,
              11: 0.0}))

In [78]:
read_files('roberta-base/newprompt', 'trec', roberta_path, multi=3)

(42, 5) 14
(42, 5) 14


({-2: -0.118,
  -1: -0.04200000000000004,
  0: -0.08400000000000007,
  1: -0.09800000000000009,
  2: -0.1140000000000001,
  3: -0.05800000000000005,
  4: -0.03200000000000003,
  5: -0.10200000000000009,
  6: -0.09800000000000009,
  7: -0.1100000000000001,
  8: -0.09400000000000008,
  9: -0.134,
  10: -0.050000000000000044,
  11: -0.17800000000000005},
 2,
 defaultdict(list,
             {-2: 0.006000000000000005,
              -1: 0.03700000000000003,
              0: 0.043999999999999984,
              1: 0.01899999999999996,
              2: 0.0019999999999999463,
              3: 0.0050000000000000044,
              4: 0.05299999999999999,
              5: 0.006000000000000005,
              6: 0.0040000000000000036,
              7: 0.03899999999999998,
              8: 0.030999999999999972,
              9: 0.04299999999999998,
              10: 0.020000000000000018,
              11: 0.0040000000000000036}))

In [71]:
import re
def extract_words(text):

    words = re.findall(r'\w+', text)
    return words

In [73]:
label_text = ['Description', 'Entity', 'Expression', 'Human', 'Location', 'Number']
   
verbalizer_dict = [k.lower() for k in label_text]
verbalizer_dict

['description', 'entity', 'expression', 'human', 'location', 'number']

In [74]:
for word in extract_words('Expression Expression Exp'.lower()):
    if word in verbalizer_dict:
        tmp = word
        break
if 

SyntaxError: incomplete input (2461967731.py, line 1)

In [75]:
'Expression Expression Exp'.lower()

'expression expression exp'